In [1]:

import time
import pandas as pd
import re

import requests
import json

import mysql
import mysql.connector
import os

import datetime
import unicodedata

from dotenv import load_dotenv
load_dotenv(".env")

True

In [2]:

def get_cursor():
    # MySQLに接続
    conn = mysql.connector.connect(
        host=os.environ.get("AWS_SLOMAP_RDS_HOST"),
        user=os.environ.get("AWS_SLOMAP_RDS_USER"),
        password=os.environ.get("AWS_SLOMAP_RDS_PASSWORD"),
        database=os.environ.get("AWS_SLOMAP_RDS_DATABASE"),
    )
    return conn

conn = get_cursor()
# カーソルを取得
cursor = conn.cursor()



In [3]:
def post_line_text(message,token):
    url = "https://notify-api.line.me/api/notify"
    headers = {"Authorization" : "Bearer "+ token}
    payload = {"message" :  message}
    post = requests.post(url ,headers = headers ,params=payload) 

In [4]:
def delete_data(cnx,day):
    cursor = cnx.cursor()
    target_days_ago = datetime.date.today() - datetime.timedelta(days=day)
    target_days_ago_str = target_days_ago.strftime('%Y-%m-%d')
    target_days_ago_str
    sql = f"DELETE FROM {os.getenv('WORDPRESS_DB_TABLE')} WHERE 日付 < '{target_days_ago_str} 00:00:00';"
    cursor.execute(sql)
    cnx.commit()




prefecture_df = pd.read_csv(r'csv\pref_lat_lon.csv')

In [5]:
conn = get_cursor()
# カーソルを取得
cursor = conn.cursor()
cursor.execute('''SELECT *
               FROM halldata ''')
cols = [col[0] for col in cursor.description]
halldata_df = pd.DataFrame(cursor.fetchall(),columns=cols)
halldata_df

,id,hall_name,prefecture_name,hall_url,dmm_url,pworld_url,line_url,twitter_url,nearest_station,address,...,machine_move,dedama_share,car_parking,mobile_charging,dedama_info,access_info,longitude,latitude,anaslo_name,bicycle_parking
0,5714,SLOTBARMs,北海道,https://hall-navi.com/hole_view?hid=0650025000...,https://p-town.dmm.com/shops/hokkaido/1087,http://www.p-world.co.jp/hokkaido/slot-bar-ms.htm,,,,北海道札幌市東区北25条東15丁目15-1,...,,,,,,,NaN,NaN,SLOTBARM's,
1,10001,パーラービックスター,東京都,https://hall-navi.com/hole_view?hid=1000101000...,https://p-town.dmm.com/shops/tokyo/1025,,,,,東京都大島町元町字北の山121-5,...,,,,,,,139.371,34.7805,,
2,10002,じゃらん六三四,東京都,https://hall-navi.com/hole_view?hid=1000211000...,https://p-town.dmm.com/shops/tokyo/1026,http://www.p-world.co.jp/tokyo/jaran634.htm,https://line.me/R/ti/p/526Coevjtz,,,東京都大島町差木地字沖の根1026-131,...,,,,,,,139.421,34.7066,,
3,10003,アリーナ羽伏,東京都,https://hall-navi.com/hole_view?hid=1000400000...,https://p-town.dmm.com/shops/tokyo/1027,,,,,東京都新島村字南桧山4-1,...,,,,,,,139.257,34.3772,,
4,10004,パーラーみやけ坪田店,東京都,https://hall-navi.com/hole_view?hid=1001101000...,https://p-town.dmm.com/shops/tokyo/1028,,,,,東京都三宅島三宅村坪田2987,...,,,,,,,139.549,34.0616,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7201,17236,サンシャイン一日橋店,沖縄県,https://hall-navi.com/hole_view?hid=9020073000...,https://p-town.dmm.com/shops/okinawa/11632,http://www.p-world.co.jp/okinawa/sanshine-1nic...,,,,沖縄県那覇市上間283−1,...,,,,,,,127.717,26.1967,サンシャインNEXT一日橋店,
7202,17237,サンシャイン具志川店,沖縄県,https://hall-navi.com/hole_view?hid=9042225000...,https://p-town.dmm.com/shops/okinawa/11684,http://www.p-world.co.jp/okinawa/sunshine-gusi...,,,,沖縄県うるま市喜屋武491−1,...,,,,,,,127.843,26.3609,サンシャイン具志川店,
7203,17238,アムズガーデン名護店,沖縄県,https://hall-navi.com/hole_view?hid=9050011000...,https://p-town.dmm.com/shops/okinawa/11669,http://www.p-world.co.jp/okinawa/ams-nago.htm,,,,沖縄県名護市宮里6丁目5−5,...,,,,,,,127.973,26.6028,アムズガーデン名護店,
7204,17239,ピータイム今帰仁,沖縄県,https://hall-navi.com/hole_view?hid=9050401000...,https://p-town.dmm.com/shops/okinawa/12798,http://www.p-world.co.jp/okinawa/p-time-nakiji...,,,,沖縄県国頭郡今帰仁村仲宗根358番地7,...,,,,,,,127.976,26.6887,,


In [7]:
prefecture_df

,pref_name,pref_name_en,latitude,longitude,area_name_en,area_name_jp
0,北海道,hokkaido,43.064359,141.347449,hokkaidoutouhoku,北海道東北
1,青森県,aomori,40.824294,140.740054,hokkaidoutouhoku,北海道東北
2,岩手県,iwate,39.703530,141.152667,hokkaidoutouhoku,北海道東北
3,宮城県,miyagi,38.268737,140.872183,hokkaidoutouhoku,北海道東北
4,秋田県,akita,39.718175,140.103356,hokkaidoutouhoku,北海道東北
5,山形県,yamagata,38.240127,140.362533,hokkaidoutouhoku,北海道東北
6,福島県,fukushima,37.750146,140.466754,hokkaidoutouhoku,北海道東北
7,茨城県,ibaraki,36.341817,140.446796,kitakantou,北関東
8,栃木県,tochigi,36.565750,139.883526,kitakantou,北関東
9,群馬県,gunma,36.391205,139.060917,kitakantou,北関東


In [6]:
conn = get_cursor()
# カーソルを取得
cursor = conn.cursor()
sql = f'''SELECT date,prefecture
            FROM groupby_date_hall_diffcoins '''#AND (取材ランク = 'S' OR 取材ランク = 'A')
print(sql)
cursor.execute(sql)
result = cursor.fetchall()
report_df = pd.DataFrame(result, columns=['イベント日','都道府県'])
report_df = report_df.astype(str)
date_list = list(report_df['イベント日'].unique())
prefecture_list = list(report_df['都道府県'].unique())
report_df_1  = report_df.drop_duplicates(subset=['イベント日','都道府県'],keep='first')
report_df_1["date_prefecture"] = report_df_1["イベント日"] + "_" +report_df_1["都道府県"]
date_prefecture_list = list(report_df_1['date_prefecture'])
print(date_prefecture_list)


line_token = os.getenv('LINE_TOKEN')


prefecture_list = list(prefecture_df['pref_name'])
#print(prefecture_list)

SELECT date,prefecture
            FROM groupby_date_hall_diffcoins 
[]


In [12]:
def removal_text(text):
    text = unicodedata.normalize("NFKC", text)
    text = text.translate(str.maketrans( '', '',string.punctuation  + '！'+ '　'+ ' '+'・'+'～' + '‐'))
    return text

def post_line_text(message,token):
    url = "https://notify-api.line.me/api/notify"
    headers = {"Authorization" : "Bearer "+ token}
    payload = {"message" :  message}
    post = requests.post(url ,headers = headers ,params=payload) 

def post_line_text_and_image(message,image_path,token):
    url = "https://notify-api.line.me/api/notify"
    headers = {"Authorization" : "Bearer "+ token}
    payload = {"message" :  message}
    #imagesフォルダの中のgazo.jpg
    print('image_path',image_path)
    files = {"imageFile":open(image_path,'rb')}
    post = requests.post(url ,headers = headers ,params=payload,files=files) 


In [14]:
date_prefecture_list

[]

In [15]:
from bs4 import BeautifulSoup
import urllib.parse
for prefecture in prefecture_list[12:13]:
    try:
        prefecture_url = urllib.parse.quote(prefecture)
        url = f'https://{os.getenv("SCRAPING_DOMAIN")}/%e3%83%9b%e3%83%bc%e3%83%ab%e3%83%87%e3%83%bc%e3%82%bf/{prefecture_url}'
        res = requests.get(url)#class="hall-list-table"
        soup = BeautifulSoup(res.text, 'lxml')

        table_elem = soup.find('div',class_='hall-list-table')
        time.sleep(1)
        tenpo_url_name_list = []
        tenpo_url_name_dict = {}
        for table_row in table_elem.find_all('div',class_='table-row'):
            try:
                hall_name = table_row.find('div',class_='table-data-cell').a.text
                url = table_row.find('div',class_='table-data-cell').a.get("href")
                #print(hall_name,url)
                tenpo_url_name = urllib.parse.unquote(url).split('/')[-2].replace('-データ一覧','')
                tenpo_url_name_list.append(tenpo_url_name)
                tenpo_url_name_dict[tenpo_url_name] = hall_name
            except:
                pass
        i = 0
        #break
        count = 0
        error_count = 0
        post_line_text(f'{prefecture} {len(tenpo_url_name_list)}店舗取得数',line_token)
        for day_num in reversed(range(1,25)):
            ichiran_all_tennpo_df = pd.DataFrame(columns=[],index=[])
            concat_df = pd.DataFrame(columns=[],index=[])
        #tenpo_ichiran_df['ホール名']
            try:
                target_day = datetime.date.today() + datetime.timedelta(days=-day_num)
                target_day_str = target_day.strftime('%Y-%m-%d')
                target_date_prefecture_str = target_day_str + '_' + prefecture
                target_date_prefecture_str
                if target_date_prefecture_str in date_prefecture_list:
                    continue
                # if i> 2:
                #     break
                for i, tenpo_name in enumerate(tenpo_url_name_list[70:83]):
                    try:
                        # if i > 5:
                        #     break
                        print(i,tenpo_name,target_day.strftime('%Y-%m-%d'))
                        url = f'https://{os.getenv("SCRAPING_DOMAIN")}/{target_day.strftime("%Y-%m-%d")}-{tenpo_name}'
                        res = requests.get(url)
                        soup = BeautifulSoup(res.text, 'html.parser')
                        try:
                            hall_name = soup.title.text.split(' ')[1]
                        except:
                            hall_name = soup.title.text
                        table = soup.find(id = "all_data_table")
                        dfs =pd.read_html(str(table))
                        #display(tenpo_df)
                        #time.sleep(1)
                        for df in  dfs:
                            try:
                                if '機種名' in list(df.columns):
                                    df['date'] = target_day.strftime('%Y-%m-%d')
                                    hall_name_text = ''
                                    for t in soup.h1.text.split(' ')[1:-1]:
                                        hall_name_text += t 

                                    df['hall_name'] = hall_name_text
                                    #print(tenpo_name)

                                    df['prefecture'] = prefecture
                                    extract_halldata_row = halldata_df[halldata_df['anaslo_name'] == hall_name_text]
                                    extract_halldata_row
                                    try:
                                        hall_id = extract_halldata_row['id'].values[0]
                                        hallnavi_name = extract_halldata_row['hall_name'].values[0]
                                    except:
                                        print('見つかりませんでした',i,tenpo_name)
                                        hall_id = ''
                                        hallnavi_name = ''
                                    df['hall_id'] = hall_id
                                    df['hallnavi_name'] = hallnavi_name
                                    concat_df = pd.concat([concat_df, df])
                                    gruopby_diff_coins_df = df.groupby(['hall_name','date']).sum().reset_index()
                                    gruopby_diff_coins_df = gruopby_diff_coins_df
                                    gruopby_diff_coins_df['hall_name'] = hall_name
                                    gruopby_diff_coins_df['hallnavi_name'] = hallnavi_name
                                    gruopby_diff_coins_df['hall_id'] = hall_id
                                    gruopby_diff_coins_df['prefecture'] = prefecture
                                    gruopby_diff_coins_df['url_hall_name'] = tenpo_name
                                    gruopby_diff_coins_df['win_machine_count'] = len(df[df['差枚'] > 0])
                                    gruopby_diff_coins_df['win_machine_count'] = gruopby_diff_coins_df['win_machine_count'].astype(str)
                                    gruopby_diff_coins_df['sum_machine_count'] = df.groupby(['hall_name','date']).size().reset_index()[0]
                                    gruopby_diff_coins_df['ave_diff_coins'] = gruopby_diff_coins_df['差枚'] / gruopby_diff_coins_df['sum_machine_count']
                                    gruopby_diff_coins_df['ave_game_count'] = gruopby_diff_coins_df['G数'] / gruopby_diff_coins_df['sum_machine_count']
                                    gruopby_diff_coins_df['sum_machine_count'] = gruopby_diff_coins_df['sum_machine_count'].astype(str)
                                    gruopby_diff_coins_df['win_rate'] = gruopby_diff_coins_df['win_machine_count'] + '/' + gruopby_diff_coins_df['sum_machine_count']
                                    gruopby_diff_coins_df['win_rate'] = gruopby_diff_coins_df['win_rate'].map(lambda x : '(' + x + '台)' + str(round(int(x.split('/')[0])/int(x.split('/')[1])*100,1))  + '%')
                                    gruopby_diff_coins_df = gruopby_diff_coins_df.fillna(0)
                                    gruopby_diff_coins_df = gruopby_diff_coins_df.astype({'ave_diff_coins':int,'ave_game_count':int})
                                    gruopby_diff_coins_df = gruopby_diff_coins_df.sort_values('date',ascending=False)
                                    gruopby_diff_coins_df = gruopby_diff_coins_df.reset_index(drop=True)
                                    
                                    gruopby_diff_coins_df = gruopby_diff_coins_df[['date','prefecture','hall_id','hall_name','hallnavi_name','url_hall_name','差枚','ave_diff_coins','ave_game_count','win_rate']]
                                    gruopby_diff_coins_df.columns = ['date','prefecture','hall_id','hall_name','hallnavi_name','url_hall_name','sum_diffcoins','ave_diffcoins','ave_game','win_rate']
                                    ichiran_all_tennpo_df =  pd.concat([ichiran_all_tennpo_df, gruopby_diff_coins_df])
                                    print('成功',i,tenpo_name)
                                    break
                                else:
                                    print('見つかりませんでした',i,tenpo_name)
                                count += 1
                                time.sleep(1)
                            except Exception as e:
                                print(tenpo_name,e)
                                error_count += 1
                                #time.sleep(1)
                                break
                                continue

                    except Exception as e:
                        print(tenpo_name,e)
                        error_count += 1
                        continue
                break
            except Exception as e:
                print(tenpo_name,e)
                continue

    except Exception as e:
        print(e)
        break
        #post_line_text(f'{prefecture}MYSQL追加処理でエラーが発生しました',line_token)
        #continue


#ichiran_all_tennpo_df.to_csv('csv/tokyo_psgr_insert_df.csv',index=False)
concat_df.rename(columns={'機種名':'machine_name',
                                                    '台番号':'machine_num',
                                                    'G数':'game_count',
                                                    '差枚':'diff_coins',
                                                    'BB':'bb_count',
                                                    'RB':'rb_count',
                                                    'ART':'art_count',
                                                    '合成確率':'sum_win_rate',
                                                    'BB確率':'bb_win_rate',
                                                    'RB確率':'rb_win_rate',
                                                    'ART確率':'art_win_rate'},inplace=True)
concat_df = concat_df.fillna(0)


0 オゼック羽村 2023-12-21
オゼック羽村 No tables found
1 オリエンタルパサージュ自由が丘 2023-12-21
成功 1 オリエンタルパサージュ自由が丘
2 オリエンタルパサージュ荻窪426 2023-12-21
成功 2 オリエンタルパサージュ荻窪426
3 オリエントヴィーナスホール 2023-12-21
成功 3 オリエントヴィーナスホール
4 オリエント秋川店 2023-12-21
成功 4 オリエント秋川店
5 ガイアネクスト三軒茶屋i 2023-12-21
成功 5 ガイアネクスト三軒茶屋i
6 ガイアネクスト三軒茶屋ii 2023-12-21
成功 6 ガイアネクスト三軒茶屋ii
7 ガイアネクスト東小金井店 2023-12-21
見つかりませんでした 7 ガイアネクスト東小金井店
成功 7 ガイアネクスト東小金井店
8 ガイア大塚北口店 2023-12-21
見つかりませんでした 8 ガイア大塚北口店
成功 8 ガイア大塚北口店
9 ガイア大島店 2023-12-21
見つかりませんでした 9 ガイア大島店
成功 9 ガイア大島店
10 ガイア大泉学園店 2023-12-21
見つかりませんでした 10 ガイア大泉学園店
成功 10 ガイア大泉学園店
11 ガイア富士見台南口店 2023-12-21
見つかりませんでした 11 ガイア富士見台南口店
成功 11 ガイア富士見台南口店
12 ガイア曳舟 2023-12-21
見つかりませんでした 12 ガイア曳舟
成功 12 ガイア曳舟


In [16]:
def insert_data_bulk(table_name,df,conn):
    sql = str(tuple(df.columns)).replace("'","") + ' values (' +  (len(df.columns) * '%s,').rstrip(',') + ')'
    insert_sql = f"""INSERT INTO {table_name} {sql} """
    print(insert_sql)
    print(ichiran_all_tennpo_df.values.tolist())
    cur = conn.cursor()
    cur.executemany(insert_sql, df.values.tolist())
    print("Insert bulk data")
    conn.commit()

In [17]:
ichiran_all_tennpo_df

,date,prefecture,hall_id,hall_name,hallnavi_name,url_hall_name,sum_diffcoins,ave_diffcoins,ave_game,win_rate
0,2023-12-21,東京都,10229,オリエンタルパサージュ自由が丘,オリエンタルパサージュ自由が丘店,オリエンタルパサージュ自由が丘,-7098,-55,455,(22/128台)17.2%
0,2023-12-21,東京都,15445,オリエンタルパサージュ荻窪426,オリエンタルパサージュ荻窪426,オリエンタルパサージュ荻窪426,-10435,-49,491,(38/211台)18.0%
0,2023-12-21,東京都,10462,オリエントヴィーナスホール,オリエントヴィーナスホール,オリエントヴィーナスホール,-9813,-45,1663,(72/218台)33.0%
0,2023-12-21,東京都,10454,オリエント秋川店,パーラーオリエント秋川店,オリエント秋川店,-19386,-184,942,(23/105台)21.9%
0,2023-12-21,東京都,10238,ガイアネクスト三軒茶屋I,ガイアネクスト三軒茶屋Ｉ,ガイアネクスト三軒茶屋i,-36690,-161,1324,(52/227台)22.9%
0,2023-12-21,東京都,10234,ガイアネクスト三軒茶屋II,ガイアネクスト三軒茶屋II,ガイアネクスト三軒茶屋ii,-35581,-184,854,(39/193台)20.2%
0,2023-12-21,東京都,,ガイアネクスト東小金井店,,ガイアネクスト東小金井店,-26759,-158,842,(26/169台)15.4%
0,2023-12-21,東京都,,ガイア大塚北口店,,ガイア大塚北口店,-21785,-76,1009,(58/286台)20.3%
0,2023-12-21,東京都,,ガイア大島店,,ガイア大島店,-21765,-102,1686,(67/212台)31.6%
0,2023-12-21,東京都,,ガイア大泉学園店,,ガイア大泉学園店,-20925,-99,685,(38/210台)18.1%


In [18]:
ichiran_all_tennpo_df.reset_index(drop=True,inplace=True)
groupby_date_hall_diffcoins_df = ichiran_all_tennpo_df
groupby_date_hall_diffcoins_df
table_name = 'groupby_date_hall_diffcoins'
insert_data_bulk(table_name,groupby_date_hall_diffcoins_df,conn)

INSERT INTO groupby_date_hall_diffcoins (date, prefecture, hall_id, hall_name, hallnavi_name, url_hall_name, sum_diffcoins, ave_diffcoins, ave_game, win_rate) values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s) 
[['2023-12-21', '東京都', 10229, 'オリエンタルパサージュ自由が丘', 'オリエンタルパサージュ自由が丘店', 'オリエンタルパサージュ自由が丘', -7098, -55, 455, '(22/128台)17.2%'], ['2023-12-21', '東京都', 15445, 'オリエンタルパサージュ荻窪426', 'オリエンタルパサージュ荻窪426', 'オリエンタルパサージュ荻窪426', -10435, -49, 491, '(38/211台)18.0%'], ['2023-12-21', '東京都', 10462, 'オリエントヴィーナスホール', 'オリエントヴィーナスホール', 'オリエントヴィーナスホール', -9813, -45, 1663, '(72/218台)33.0%'], ['2023-12-21', '東京都', 10454, 'オリエント秋川店', 'パーラーオリエント秋川店', 'オリエント秋川店', -19386, -184, 942, '(23/105台)21.9%'], ['2023-12-21', '東京都', 10238, 'ガイアネクスト三軒茶屋I', 'ガイアネクスト三軒茶屋Ｉ', 'ガイアネクスト三軒茶屋i', -36690, -161, 1324, '(52/227台)22.9%'], ['2023-12-21', '東京都', 10234, 'ガイアネクスト三軒茶屋II', 'ガイアネクスト三軒茶屋II', 'ガイアネクスト三軒茶屋ii', -35581, -184, 854, '(39/193台)20.2%'], ['2023-12-21', '東京都', '', 'ガイアネクスト東小金井店', '', 'ガイアネクスト東小金井店', -26759, -158, 842, '(26/169台)15

In [22]:


#daily_record用のデータフレーム作成
daily_record_df = concat_df.reset_index(inplace=False,drop=False)
daily_record_df=daily_record_df.rename(columns={'index':'id'})
daily_record_df = daily_record_df[['hall_id','date', 'hall_name','hallnavi_name' , 'prefecture',
        'machine_name', 'machine_num', 'game_count', 'diff_coins',
       'bb_count', 'rb_count', 'art_count', 'sum_win_rate', 'bb_win_rate',
       'rb_win_rate', 'art_win_rate' ]]
groupby_date_machine_number_diffcoins_df = daily_record_df
groupby_date_machine_number_diffcoins_df
table_name = 'groupby_date_machine_number_diffcoins'
insert_data_bulk(table_name,groupby_date_machine_number_diffcoins_df,conn)

concat_groupby_date_kisyubetu_df = pd.DataFrame(columns=[],index=[])
for hall_name in concat_df['hall_name'].unique():
    extract_hall_status_df= concat_df[concat_df['hall_name'] == hall_name]
    #display(extract_hall_status_df)
    hall_id = extract_hall_status_df['hall_id'].values[0]
    groupby_date_kisyubetu_df = extract_hall_status_df.groupby(['date','machine_name']).sum()
    groupby_date_kisyubetu_df['sum_machine_count'] = extract_hall_status_df.groupby(['date','machine_name']).size()
    groupby_date_kisyubetu_df = groupby_date_kisyubetu_df.reset_index(drop=False).reset_index().rename(columns={'index': 'ranking'})
    groupby_date_kisyubetu_df[['date','machine_name','sum_machine_count','game_count','diff_coins']]
    groupby_date_kisyubetu_df['hall_name'] = hall_name
    hall_id = extract_hall_status_df['hall_id'].values[0]
    hallnavi_name = extract_hall_status_df['hall_name'].values[0]
    kisyubetu_win_daissuu_list = []
    groupby_date_kisyubetu_df_list = []
    for kisyu_name,day in zip(groupby_date_kisyubetu_df['machine_name'],groupby_date_kisyubetu_df['date']):
        kisyu_df = extract_hall_status_df.query('machine_name == @kisyu_name & date == @day')
        groupby_date_kisyubetu_df_list.append(kisyu_df)
        kisyu_win_daisuu = len(kisyu_df[kisyu_df['diff_coins'] > 0])
        kisyubetu_win_daissuu_list.append(kisyu_win_daisuu)
    groupby_date_kisyubetu_df['win_rate'] = kisyubetu_win_daissuu_list
    groupby_date_kisyubetu_df['win_machine_count'] = kisyubetu_win_daissuu_list
    groupby_date_kisyubetu_df['win_rate'] = groupby_date_kisyubetu_df['win_rate'].astype(str)
    groupby_date_kisyubetu_df['sum_machine_count'] = groupby_date_kisyubetu_df['sum_machine_count'].astype(int)
    groupby_date_kisyubetu_df['ave_game_count'] = groupby_date_kisyubetu_df['game_count'] / groupby_date_kisyubetu_df['sum_machine_count']
    groupby_date_kisyubetu_df['ave_game_count'] = groupby_date_kisyubetu_df['ave_game_count'].astype(int)
    groupby_date_kisyubetu_df = groupby_date_kisyubetu_df[groupby_date_kisyubetu_df['sum_machine_count'] >= 2 ]
    groupby_date_kisyubetu_df['diff_coins'] = groupby_date_kisyubetu_df['diff_coins'].astype(int)
    groupby_date_kisyubetu_df['ave_diff_coins'] = groupby_date_kisyubetu_df['diff_coins'] / groupby_date_kisyubetu_df['sum_machine_count']
    groupby_date_kisyubetu_df['ave_diff_coins'] = groupby_date_kisyubetu_df['ave_diff_coins'].astype(int)
    groupby_date_kisyubetu_df['sum_machine_count'] = groupby_date_kisyubetu_df['sum_machine_count'].astype(str)
    groupby_date_kisyubetu_df['win_rate'] = groupby_date_kisyubetu_df['win_rate'] + '/' + groupby_date_kisyubetu_df['sum_machine_count']
    groupby_date_kisyubetu_df['win_rate'] = groupby_date_kisyubetu_df['win_rate'].map(lambda x : '(' + x + '台) ' + str(round(int(x.split('/')[0])/int(x.split('/')[1])*100,1))  + '%')
    groupby_date_kisyubetu_df = groupby_date_kisyubetu_df[['date','ranking','machine_name','win_rate','ave_game_count','ave_diff_coins','diff_coins','game_count','win_machine_count','sum_machine_count']]
    groupby_date_kisyubetu_df = groupby_date_kisyubetu_df.sort_values('ave_diff_coins',ascending=False)
    groupby_date_kisyubetu_df['machine_ave_rate'] =(((groupby_date_kisyubetu_df['game_count'] * 3) + groupby_date_kisyubetu_df['diff_coins']) / (groupby_date_kisyubetu_df['game_count'] * 3) )*100
    groupby_date_kisyubetu_df['machine_ave_rate'] = groupby_date_kisyubetu_df['machine_ave_rate'].map(lambda x : round(x,1))
    groupby_date_kisyubetu_df['machine_ave_rate'] = groupby_date_kisyubetu_df['machine_ave_rate'].fillna(0)
    groupby_date_kisyubetu_df = groupby_date_kisyubetu_df.rename(columns={'game_count': 'sum_game_count','diff_coins': 'sum_diff_coins'})
    groupby_date_kisyubetu_df['hall_name'] = hall_name
    groupby_date_kisyubetu_df['hall_id'] = hall_id
    groupby_date_kisyubetu_df['hallnavi_name'] = hallnavi_name
    groupby_date_kisyubetu_df = groupby_date_kisyubetu_df[['hall_id','hall_name','hallnavi_name','date','machine_name','win_rate','machine_ave_rate','ave_game_count','ave_diff_coins','sum_game_count','sum_diff_coins','win_machine_count','sum_machine_count']]
    concat_groupby_date_kisyubetu_df = pd.concat([concat_groupby_date_kisyubetu_df, groupby_date_kisyubetu_df])
    
concat_groupby_date_kisyubetu_df.reset_index(drop=True,inplace=True)
groupby_date_machine_diffcoins_df = concat_groupby_date_kisyubetu_df
groupby_date_machine_diffcoins_df
table_name = 'groupby_date_machine_diffcoins'
insert_data_bulk(table_name,groupby_date_machine_diffcoins_df,conn)

INSERT INTO groupby_date_machine_number_diffcoins (hall_id, date, hall_name, hallnavi_name, prefecture, machine_name, machine_num, game_count, diff_coins, bb_count, rb_count, art_count, sum_win_rate, bb_win_rate, rb_win_rate, art_win_rate) values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s) 
[['2023-12-20', '東京都', 10160, 'TOYO', 'TOYOHALL', 'toyo-hall', -17638, -153, 1489, '(32/115台)27.8%'], ['2023-12-20', '東京都', 10027, 'ピーアーク三田', 'ピーアーク三田', 'ピーアーク三田', -35049, -190, 1800, '(64/184台)34.8%'], ['2023-12-20', '東京都', 10165, '123+N東雲店', '123+N東雲店', '123n東雲店', -33428, -86, 1691, '(98/387台)25.3%'], ['2023-12-20', '東京都', 10236, 'AQUA', 'アクアH2O(AQUA)', 'aqua-h2o', -9788, -152, 848, '(16/64台)25.0%']]
Insert bulk data
INSERT INTO groupby_date_machine_diffcoins (hall_id, hall_name, hallnavi_name, date, machine_name, win_rate, machine_ave_rate, ave_game_count, ave_diff_coins, sum_game_count, sum_diff_coins, win_machine_count, sum_machine_count) values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s) 
[

In [ ]:
!pip install git+https://github.com/dataanalytics2020/nuxt3_fastapi_aws

In [7]:
from github import Github
# Authentication is defined via github.Auth
from github import Auth
import os
import json
import requests
from dotenv import load_dotenv
load_dotenv(".env")
# ユーザ名、パスワードによるインスタンス生成
GITHUB_PERSONAL_ACCESS_TOKENS = os.environ.get('GITHUB_PERSONAL_ACCESS_TOKENS')
# using an access token
auth = Auth.Token(GITHUB_PERSONAL_ACCESS_TOKENS)

# First create a Github instance:

# Public Web Github
g = Github(auth=auth)
for repo in g.get_user().get_repos():
    print(repo)

Repository(full_name="dataanalytics2020/100knocks-preprocess-1")
Repository(full_name="dataanalytics2020/akasaka_blot_post_toukai")
Repository(full_name="dataanalytics2020/create_tomorrow_birthday_image_post_twitter")
Repository(full_name="dataanalytics2020/django_blog")
Repository(full_name="dataanalytics2020/django_portfolio")
Repository(full_name="dataanalytics2020/docker_django")
Repository(full_name="dataanalytics2020/docker_django_project")
Repository(full_name="dataanalytics2020/drf_vue_website")
Repository(full_name="dataanalytics2020/fastapi_practice")
Repository(full_name="dataanalytics2020/flask_website")
Repository(full_name="dataanalytics2020/jupyter_javascript")
Repository(full_name="dataanalytics2020/microCMS-blog")
Repository(full_name="dataanalytics2020/mysql_sqlalchemy_practice")
Repository(full_name="dataanalytics2020/nuxt3_fastapi_aws")
Repository(full_name="dataanalytics2020/practice_multiple_regression_analysis")
Repository(full_name="dataanalytics2020/react_djang